In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [ ]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
def read_data(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [ ]:
def combine_DF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [ ]:
def select_farmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df = df[(df.交易標的 == "土地")]  
    df_farmland = df[(((df.非都市土地使用分區 == '特定農業區')|(df.非都市土地使用分區 == "一般農業區"))&(df.非都市土地使用編定 =="農牧用地"))]
    #排除都市土地
    df_farmland = df_farmland[df_farmland["都市土地使用分區"].isnull()]
    return df_farmland

In [ ]:
def fill_nan_price(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculate_price(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [ ]:
def calculate_price(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [ ]:
def calculate_mean(df_farmland):

    landpriceMean = pd.to_numeric(df_farmland['單價(元/平方公尺)']).mean()
   
    return landpriceMean

In [ ]:
def grouptown_calculate_mean(df,list_all_town):
    
    grouped=pd.to_numeric(df['單價(元/平方公尺)']).groupby(df['鄉鎮市區'])
    town_dict = {}
    for key,group in grouped:  
        m = grouped.mean()
        town_dict[key]= m[key]*3.3058 #1ping = 3.3058 m^2
        if key not in list_all_town:
            list_all_town.append(key)
    return {"towndict":town_dict,"town_list":list_all_town}

In [ ]:
def calculate_gain(cityMeandict,div):
    
    #dict to df
    arr = list(cityMeandict.items())
    df = pd.DataFrame(arr, columns=['年份','單價'])
    
    #逐年漲幅
    gain = df["單價"].pct_change(div)
    
    return gain

In [ ]:
def group_data_per_town(city,start_year,end_year,start_season,end_season):
    s = start_season
    df = pd.DataFrame()
    for y in range(start_year,end_year+1): #every year
         
        while s <= 4:
            # read data
            df_s =  read_data(str(y),str(s),city)
            
            if city=='Q':# combine嘉義縣市(Q,I) 
                df_s = combine_DF(df_s,read_data(str(y),str(s),'I'))
            if city=='J':# combine新竹縣市(J,O)   
                df_s = combine_DF(df_s,read_data(str(y),str(s),'O'))
               
            df = combine_DF(df,df_s)
            
            if y == end_year and s == end_season:
                break
            s = s+1
        s=1
        
    # select farmland
    df_farmland = select_farmland(df)
    # fill nan price
    df_farmland = fill_nan_price(df_farmland)
    return df_farmland

In [ ]:
#exclude the outer land and combine 嘉義 新竹縣市
list_cityfilename = ['A','B','C','D','E','F','G','H','J','K','M','N','P','Q','T','U','V']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','新竹縣','苗栗縣','南投縣','彰化縣','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣']

START_SEASON = 4
END_SEASON = 2
START_YEAR = 101
END_YEAR = 107
all_city_price_dict = {}

i=0
for c in list_cityfilename: 
    s = START_SEASON 
    city_mean_dict = {} 
    town_mean_dict = {} #各鄉鎮逐年平均
    list_all_town = [] #該城市的鄉鎮列表
    
    for y in range(START_YEAR,END_YEAR+1): 
        
        df = pd.DataFrame()
        
        while s <= 4:
            # read data
            df_s =  read_data(str(y),str(s),c)
            
            if c=='Q':# combine嘉義縣市(Q,I) 
                df_s = combine_DF(df_s,read_data(str(y),str(s),'I'))
            if c=='J':# combine新竹縣市(J,O)   
                df_s = combine_DF(df_s,read_data(str(y),str(s),'O'))
            
            df = combine_DF(df,df_s)
           
            if y == END_YEAR and s == END_SEASON :
                break
            s = s+1
        s=1
       
        # select farmland
        df_farmland = select_farmland(df)
        # fill nan price
        df_farmland = fill_nan_price(df_farmland)
        
        # 縣市年的平均
        mean = calculate_mean(df_farmland)
        city_mean_dict [str(y)] = mean*3.3058 #1ping = 3.3058 m^2
        
        result = grouptown_calculate_mean(df_farmland,list_all_town)
        town_mean_dict[str(y)] = result["towndict"]
       
    all_city_price_dict[list_city[i]] = city_mean_dict
    
    #各鄉鎮逐年單價排名
    print(list_city[i])
    print("各鄉鎮逐年單價排名:")
    table_dict = {}
    for y in range(101,108):
        sorted_town = sorted(town_mean_dict[str(y)].items(),key = lambda s:s[1],reverse=True)
        table_dict[y] = sorted_town
   
    print("|排名\年",end='')
    for y in range(101,108):
        print("|"+str(y),end='')
    print("|")
    for y in range(101,108):
        print("|------",end='')
    print("|")
    for t in range(len(list_all_town)):
        print("|"+str(t+1)+"|",end='')
        for y in range(101,108):
            if t < len(table_dict[y]):
                print(table_dict[y][t][0]+" / "+"{:.1f}".format(table_dict[y][t][1])+"|",end="") 
            else:
                print("-|",end="")
        print("")
        
    
        
    i+=1
print(all_city_price_dict) 

In [ ]:
#exclude the outer land and combine 嘉義 新竹縣市
list_cityfilename = ['A','B','C','D','E','F','G','H','J','K','M','N','P','Q','T','U','V']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','新竹縣','苗栗縣','南投縣','彰化縣','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣']

START_SEASON = 4
END_SEASON = 2
START_YEAR = 101
END_YEAR = 107
all_city_price_dict = {}
i=0
for c in list_cityfilename: 
    s = START_SEASON 
    city_mean_dict = {} 
   
    for y in range(START_YEAR,END_YEAR+1): 
        
        while s <= 4:
            df = pd.DataFrame()
            # read data
            df_s =  read_data(str(y),str(s),c)
            
            if c=='Q':# combine嘉義縣市(Q,I) 
                df_s = combine_DF(df_s,read_data(str(y),str(s),'I'))
            if c=='J':# combine新竹縣市(J,O)   
                df_s = combine_DF(df_s,read_data(str(y),str(s),'O'))
               
            df = combine_DF(df,df_s)
            
            # select farmland
            df_farmland = select_farmland(df)
            # fill nan price
            df_farmland = fill_nan_price(df_farmland)
        
            # 縣市年的平均
            
            mean = calculate_mean(df_farmland)
            city_mean_dict [str(y)+"-"+str(s)] =mean*3.3058 #1ping = 3.3058 m^2
            
            if y == END_YEAR and s == END_SEASON :
                break
            s = s+1
        s=1
        
    all_city_price_dict[list_city[i]] = city_mean_dict
    i+=1
print(all_city_price_dict)   

In [ ]:
# early year from 101-s4 to 102-s3 
# recent year from 106-s3 to 107-s2
i=0
town_price = {}
for c in list_cityfilename: #every city
    
    city_mean_dict = {} #各城市逐年平均
    town_mean_dict = {} #各鄉鎮逐年平均
    list_all_town = [] #該城市的鄉鎮列表
    
    df_farmland = group_data_per_town(c,101,102,4,3)    
    # 縣市年的平均
    mean = calculate_mean(df_farmland)
    city_mean_dict[0] = mean*3.3058 #1ping = 3.3058 m^2
    # 鄉鎮平均
    result = grouptown_calculate_mean(df_farmland,list_all_town)
    town_mean_dict[0] = result["towndict"]
     
    
    df_farmland = group_data_per_town(c,102,103,4,3)  
    # 鄉鎮平均
    result = grouptown_calculate_mean(df_farmland,list_all_town)
    town_mean_dict[1] = result["towndict"]
    
    df_farmland = group_data_per_town(c,103,104,4,3)  
    # 鄉鎮平均
    result = grouptown_calculate_mean(df_farmland,list_all_town)
    town_mean_dict[2] = result["towndict"]
    
    df_farmland = group_data_per_town(c,104,105,4,3)  
    # 鄉鎮平均
    result = grouptown_calculate_mean(df_farmland,list_all_town)
    town_mean_dict[3] = result["towndict"]
    
    df_farmland = group_data_per_town(c,105,106,3,2)  
    # 鄉鎮平均
    result = grouptown_calculate_mean(df_farmland,list_all_town)
    town_mean_dict[4] = result["towndict"]
    
    df_farmland = group_data_per_town(c,106,107,3,2)   
    # 縣市年的平均
    mean = calculate_mean(df_farmland)
    city_mean_dict[1] = mean*3.3058 #1ping = 3.3058 m^2
    
    # 鄉鎮平均
    result = grouptown_calculate_mean(df_farmland,list_all_town)
    town_mean_dict[5] = result["towndict"]

    town_price[list_city[i]] = city_mean_dict
    
    #鄉鎮總漲幅
    print(list_city[i])
    print("鄉鎮總漲幅:")
    鄉鎮逐年平均單價={}
    鄉鎮歷年平均單價 = {}
    鄉鎮年度漲幅 = {}
    鄉鎮逐年漲幅 = {}
    for t in list_all_town:
        鄉鎮逐年平均單價 = {}
        for y in range(0,6):
            
            if t in town_mean_dict[y].keys():
                鄉鎮逐年平均單價[y] = town_mean_dict[y][t]
        鄉鎮歷年平均單價[t] = 鄉鎮逐年平均單價
    
        gain = calculate_gain(鄉鎮歷年平均單價[t],len(鄉鎮歷年平均單價[t].keys())-1)
        for g in gain:
            if np.isnan(g) == False:
                 鄉鎮年度漲幅[t] = g
    print( 鄉鎮歷年平均單價)
    sorted_gain = sorted(鄉鎮年度漲幅.items(),key=lambda s:s[1],reverse=True)

    print("|排名\年|漲幅|")

    print("|------|------|")
    for h in range(len(鄉鎮年度漲幅.keys())):
        print("|"+str(h+1)+"|",end="")  
        print(sorted_gain[h][0]+"/"+"{:.1f}".format(sorted_gain[h][1]*100)+"|")     
    i+=1

In [ ]:
#city total gain
print("city total gain:")
town_gain={}
for t in list_city:
    gain = calculate_gain(town_price[t],1)
    
    total_gain = {}
    for g in gain:
        if np.isnan(g) == False:
            town_gain[t] = g    
        
sorted_gain = sorted(town_gain.items(),key=lambda s:s[1],reverse=True)
print("|排名\年|漲幅|")

print("|------|------|")

for i in range(0,15):
    if np.isnan(sorted_gain[i][1]) == False:
        print("|"+str(i+1),end="")
        print("|"+sorted_gain[i][0]+"/"+ "{:.1f}".format(sorted_gain[i][1]*100)+"|")